In [1]:
import os
%pwd

'c:\\Users\\Akash R\\OneDrive\\Desktop\\Projectts\\End-to-End-Loan-Default-Prediction\\NoteBook_Experiments'

In [2]:
os.chdir('../')
%pwd

'c:\\Users\\Akash R\\OneDrive\\Desktop\\Projectts\\End-to-End-Loan-Default-Prediction'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from LoanDefaultClassifier.constants import *
from LoanDefaultClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config


In [6]:
#Components
import os
from LoanDefaultClassifier import logger
import urllib.request as request
import zipfile
from LoanDefaultClassifier.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloading {filename}!")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config =  config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-01 02:17:02,818: INFO: common: YAML file: config\config.yaml loaded successfully]
[2024-01-01 02:17:02,847: INFO: common: YAML file: params.yaml loaded successfully]
[2024-01-01 02:17:02,851: INFO: common: created directory at: artifacts]
[2024-01-01 02:17:02,855: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-01 02:17:07,042: INFO: 183055017: Downloading artifacts/data_ingestion/data.zip with the following info: 
Connection: close
Content-Length: 2938118
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "69a9f49226590e831bcedf6430faf4bae6a84e4e0a5f0a74401351e65ec27901"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7E20:10A8:7EA3F0:9DB0E7:6591D347
Accept-Ranges: bytes
Date: Sun, 31 Dec 2023 20:47:06 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10250-MAA
X-Cache